In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import sys
print (sys.version)

2.7.12 |Anaconda 4.2.0 (64-bit)| (default, Jun 29 2016, 11:07:13) [MSC v.1500 64 bit (AMD64)]


In [2]:
import pandas as pd
import smtplib #simple mail transfer protocol
import mimetypes #for guessing the mime type

#import the email modules
import email
import email.mime.application

In [3]:
#import the clearing and issuing EEA Definitions for MC and VI
issue=pd.read_csv('G:\\Pricing\\AIB\\visa_mastercard_eea_issuing_definitions.csv')
clear=pd.read_csv('G:\\Pricing\\AIB\\visa_mastercard_eea_clearing_definitions.csv')

In [4]:
issue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 3 columns):
CARD_SCHEME                 97 non-null object
ISSUER_COUNTRY              97 non-null object
EUROPE_ISSUING_SUBREGION    97 non-null object
dtypes: object(3)
memory usage: 2.3+ KB


In [5]:
clear.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 3 columns):
CARD_SCHEME                  97 non-null object
MERCHANT_COUNTRY             97 non-null object
EUROPE_CLEARING_SUBREGION    97 non-null object
dtypes: object(3)
memory usage: 2.3+ KB


In [6]:
columns=['CONTRACT_ID','ISSUER_COUNTRY','MERCHANT_COUNTRY','REGIONALITY','BIN_DESC','CARD_SCHEME','description','SECURITY_LEVEL','AIB_SF_RATE_AMOUNT','AIB_SF_RATE_PERC','ROUNDED_TRXN_AMOUNT','AIB_SF_BPS','CREDIT_DEBIT','CONSUMER_CORPORATE']

In [7]:
df=pd.read_csv('D:\\djohnson\\Desktop\\aib_comp_pull_aib_data_sf.txt',skiprows=47,error_bad_lines=False,sep=';',names=columns,low_memory=True)

C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
df.head()

,CONTRACT_ID,ISSUER_COUNTRY,MERCHANT_COUNTRY,REGIONALITY,BIN_DESC,CARD_SCHEME,description,SECURITY_LEVEL,AIB_SF_RATE_AMOUNT,AIB_SF_RATE_PERC,ROUNDED_TRXN_AMOUNT,AIB_SF_BPS,CREDIT_DEBIT,CONSUMER_CORPORATE
0,1946,Spain ...,Luxembourg ...,Intra-region ...,CLASSIC,Visa ...,VISALE_D_CardNOTPresent_Intra_EEA ...,NON-SECURE,.013672727,.0002,15.0,0.001429,Debit,Consumer
1,1946,Netherlands ...,Luxembourg ...,Intra-region ...,CLASSIC,MasterCard ...,MCSALE_C_NonSecured_Intra_EEA ...,NON-SECURE,.03,.00059,27.0,0.001876,Credit,Consumer
2,1946,Italy ...,Luxembourg ...,Intra-region ...,CLASSIC,Visa ...,VISALE_P_CardNOTPresent_Intra_EEA ...,NON-SECURE,.01374,.0002,22.0,0.001010,Prepaid,Consumer
3,1946,Bulgaria ...,Luxembourg ...,Intra-region ...,CLASSIC,Visa ...,VISALE_D_CardNOTPresent_Intra_EEA ...,NON-SECURE,.01374,.0002,7.0,0.002955,Debit,Consumer
4,1946,United Kingdom ...,Luxembourg ...,Intra-region ...,CLASSIC,Visa ...,VISALE_D_CardNOTPresent_Intra_EEA ...,NON-SECURE,.01,.0002,9.0,0.002326,Debit,Consumer


In [9]:
bb_columns=['TRANSACTION_REF','ADDITIONAL_REFERENCE_2','PAYMENTREFERENCE','BB_TRANSACTION_TYPE','MCC','CONTRACT_ID','IIN','BIN_DESC','ORDERID','PAYMENT_PROCESSOR','MONTH','DAY','YEAR','PERIOD'
              ,'ISSUER_COUNTRY','MERCHANT_COUNTRY','TRANSACTION_TYPE','CARD_SCHEME','CREDIT_DEBIT','CONSUMER_CORPORATE','SECURITY_LEVEL','REGIONALITY'
              ,'PRODUCT_CODE','SF_FIXED_FEE','SF_PERC_FEE','CON_COM_BIN','VOLUME','ROUNDED_TRXN_AMOUNT','SCHEME_TOTAL_FEE_EUR','FLOW','BB_IC_TOTAL','BB_IC_BPS','BB_SF_BPS']

In [10]:
bb=pd.read_csv('d:\\djohnson\\Desktop\\aib_comp_pull_bb_data.csv',error_bad_lines=False,sep=';',skiprows=95,names=bb_columns,index_col=False)

In [11]:
bb['BB_TRANSACTION_TYPE']=bb['BB_TRANSACTION_TYPE'].str.strip()
bb['ISSUER_COUNTRY']=bb['ISSUER_COUNTRY'].str.strip()
bb['MERCHANT_COUNTRY']=bb['MERCHANT_COUNTRY'].str.strip()
bb['CARD_SCHEME']=bb['CARD_SCHEME'].str.strip()
bb['CREDIT_DEBIT']=bb['CREDIT_DEBIT'].str.strip()
bb['CONSUMER_CORPORATE']=bb['CONSUMER_CORPORATE'].str.strip()
bb['REGIONALITY']=bb['REGIONALITY'].str.strip()
bb['BIN_DESC']=bb['BIN_DESC'].str.strip()

In [12]:
bb=bb[(bb['CONTRACT_ID']==1946)|(bb['CONTRACT_ID']==2095)|(bb['CONTRACT_ID']==6886)|(bb['CONTRACT_ID']==6891)]

In [13]:
#create a new copy and safe old in memory for working purposes and to prevent repeated import from local drive
bb1=bb.copy()
bb1=bb1.dropna()
bb1=bb1.drop(['CON_COM_BIN','BB_IC_BPS','BB_IC_TOTAL','MCC','IIN','PAYMENT_PROCESSOR','VOLUME','PRODUCT_CODE'],axis=1)

In [14]:
bb1['SECURITY_LEVEL']=bb1['SECURITY_LEVEL'].apply(lambda x:'NON-SECURE' if (7 == x) or (0 == x) or (6 == x) else 'SECURE')

In [15]:
bb1['CARD_SCHEME']=bb1['CARD_SCHEME'].apply(lambda x:'MasterCard' if ('Mastercard'==x) else 'Visa')

In [16]:
bb1=bb1[(bb1['BB_TRANSACTION_TYPE']=='Sale')]
bb1=bb1.rename({'ROUNDED_TRXN_AMOUNT':'FLOW'})
bb1.head()

,TRANSACTION_REF,ADDITIONAL_REFERENCE_2,PAYMENTREFERENCE,BB_TRANSACTION_TYPE,CONTRACT_ID,BIN_DESC,ORDERID,MONTH,DAY,YEAR,...,CREDIT_DEBIT,CONSUMER_CORPORATE,SECURITY_LEVEL,REGIONALITY,SF_FIXED_FEE,SF_PERC_FEE,ROUNDED_TRXN_AMOUNT,SCHEME_TOTAL_FEE_EUR,FLOW,BB_SF_BPS
18,225802414299,2.258024e+11,2.258024e+11,Sale,6891.0,CLASSIC,16621009.0,2.0,17.0,2018.0,...,Debit,Consumer,NON-SECURE,Intraregional,0.004896,0.000299,9.0,0.007482,8.65,0.000865
23,225958428498,2.259584e+11,2.259584e+11,Sale,6891.0,CLASSIC,16727950.0,3.0,13.0,2018.0,...,Debit,Consumer,NON-SECURE,Intraregional,0.004777,0.000299,9.0,0.007363,8.65,0.000851
24,225958428479,2.259584e+11,2.259584e+11,Sale,6891.0,CLASSIC,16727861.0,3.0,13.0,2018.0,...,Debit,Consumer,NON-SECURE,Intraregional,0.022242,0.001885,9.0,0.038548,8.65,0.004456
31,225934411851,2.259344e+11,2.259344e+11,Sale,6891.0,CLASSIC,16706067.0,3.0,9.0,2018.0,...,Credit,Consumer,NON-SECURE,Interregional,0.196247,0.005460,9.0,0.243477,8.65,0.028148
32,225958428399,2.259584e+11,2.259584e+11,Sale,6891.0,CLASSIC,16727242.0,3.0,13.0,2018.0,...,Debit,Consumer,NON-SECURE,Intraregional,0.022242,0.001460,9.0,0.035602,9.15,0.003891


In [17]:
bb1=pd.merge(bb1,issue,on=['ISSUER_COUNTRY','CARD_SCHEME'],how='left')
bb1=pd.merge(bb1,clear,on=['MERCHANT_COUNTRY','CARD_SCHEME'],how='left')

In [18]:
bb1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40597 entries, 0 to 40596
Data columns (total 27 columns):
TRANSACTION_REF              40597 non-null object
ADDITIONAL_REFERENCE_2       40597 non-null float64
PAYMENTREFERENCE             40597 non-null float64
BB_TRANSACTION_TYPE          40597 non-null object
CONTRACT_ID                  40597 non-null float64
BIN_DESC                     40597 non-null object
ORDERID                      40597 non-null float64
MONTH                        40597 non-null float64
DAY                          40597 non-null float64
YEAR                         40597 non-null float64
PERIOD                       40597 non-null object
ISSUER_COUNTRY               40597 non-null object
MERCHANT_COUNTRY             40597 non-null object
TRANSACTION_TYPE             40597 non-null object
CARD_SCHEME                  40597 non-null object
CREDIT_DEBIT                 40597 non-null object
CONSUMER_CORPORATE           40597 non-null object
SECURITY_LEVEL   

In [19]:
bb1['EEA_DESIGNATION']=''
bb1['EEA_DESIGNATION'][(bb1['EUROPE_ISSUING_SUBREGION']=='EEA')&(bb1['EUROPE_CLEARING_SUBREGION']=='EEA')]='EEA'
bb1['EEA_DESIGNATION'][(bb1['EUROPE_ISSUING_SUBREGION']!='EEA')|(bb1['EUROPE_CLEARING_SUBREGION']!='EEA')]='NON-EEA'
bb1['EUROPE_ISSUING_SUBREGION'][(bb1['EUROPE_ISSUING_SUBREGION']=='')]='NON-EEA'

C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [20]:
bb1.head()

,TRANSACTION_REF,ADDITIONAL_REFERENCE_2,PAYMENTREFERENCE,BB_TRANSACTION_TYPE,CONTRACT_ID,BIN_DESC,ORDERID,MONTH,DAY,YEAR,...,REGIONALITY,SF_FIXED_FEE,SF_PERC_FEE,ROUNDED_TRXN_AMOUNT,SCHEME_TOTAL_FEE_EUR,FLOW,BB_SF_BPS,EUROPE_ISSUING_SUBREGION,EUROPE_CLEARING_SUBREGION,EEA_DESIGNATION
0,225802414299,2.258024e+11,2.258024e+11,Sale,6891.0,CLASSIC,16621009.0,2.0,17.0,2018.0,...,Intraregional,0.004896,0.000299,9.0,0.007482,8.65,0.000865,EEA,EEA,EEA
1,225958428498,2.259584e+11,2.259584e+11,Sale,6891.0,CLASSIC,16727950.0,3.0,13.0,2018.0,...,Intraregional,0.004777,0.000299,9.0,0.007363,8.65,0.000851,EEA,EEA,EEA
2,225958428479,2.259584e+11,2.259584e+11,Sale,6891.0,CLASSIC,16727861.0,3.0,13.0,2018.0,...,Intraregional,0.022242,0.001885,9.0,0.038548,8.65,0.004456,EEA,EEA,EEA
3,225934411851,2.259344e+11,2.259344e+11,Sale,6891.0,CLASSIC,16706067.0,3.0,9.0,2018.0,...,Interregional,0.196247,0.005460,9.0,0.243477,8.65,0.028148,NaN,EEA,NON-EEA
4,225958428399,2.259584e+11,2.259584e+11,Sale,6891.0,CLASSIC,16727242.0,3.0,13.0,2018.0,...,Intraregional,0.022242,0.001460,9.0,0.035602,9.15,0.003891,EEA,EEA,EEA


In [21]:
df.MERCHANT_COUNTRY=df.MERCHANT_COUNTRY.str.strip()
df.REGIONALITY=df.REGIONALITY.str.strip()
df.CARD_SCHEME=df.CARD_SCHEME.str.strip()
df.ISSUER_COUNTRY=df.ISSUER_COUNTRY.str.strip()
df.description=df.description.str.strip()
df.BIN_DESC=df.BIN_DESC.str.strip()

df.AIB_SF_RATE_AMOUNT=df.AIB_SF_RATE_AMOUNT.str.strip()
df.AIB_SF_RATE_PERC=df.AIB_SF_RATE_PERC.str.strip()

In [22]:
df=pd.merge(df,issue,on=['ISSUER_COUNTRY','CARD_SCHEME'],how='left')
df=pd.merge(df,clear,on=['MERCHANT_COUNTRY','CARD_SCHEME'],how='left')

In [23]:
df['EEA_DESIGNATION']=''
df['EEA_DESIGNATION'][(df['EUROPE_ISSUING_SUBREGION']=='EEA')&(df['EUROPE_CLEARING_SUBREGION']=='EEA')]='EEA'
df['EEA_DESIGNATION'][(df['EUROPE_ISSUING_SUBREGION']!='EEA')|(df['EUROPE_CLEARING_SUBREGION']!='EEA')]='NON-EEA'

C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3325809 entries, 0 to 3325808
Data columns (total 17 columns):
CONTRACT_ID                  object
ISSUER_COUNTRY               object
MERCHANT_COUNTRY             object
REGIONALITY                  object
BIN_DESC                     object
CARD_SCHEME                  object
description                  object
SECURITY_LEVEL               object
AIB_SF_RATE_AMOUNT           object
AIB_SF_RATE_PERC             object
ROUNDED_TRXN_AMOUNT          float64
AIB_SF_BPS                   float64
CREDIT_DEBIT                 object
CONSUMER_CORPORATE           object
EUROPE_ISSUING_SUBREGION     object
EUROPE_CLEARING_SUBREGION    object
EEA_DESIGNATION              object
dtypes: float64(2), object(15)
memory usage: 456.7+ MB


In [25]:
df['AIB_SF_BPS']=df['AIB_SF_BPS'].dropna()
df['AIB_SF_RATE_AMOUNT']=df['AIB_SF_RATE_AMOUNT'].dropna()
df['AIB_SF_RATE_PERC']=df['AIB_SF_RATE_PERC'].dropna()

df=df[(df['AIB_SF_BPS']>0)]
df=df[(df['AIB_SF_RATE_AMOUNT']!='')]
df=df[(df['AIB_SF_RATE_PERC']!='')]

df['AIB_SF_BPS']=df['AIB_SF_BPS'].astype(float)
df['AIB_SF_RATE_AMOUNT']=df['AIB_SF_RATE_AMOUNT'].astype(float)
df['AIB_SF_RATE_PERC']=df['AIB_SF_RATE_PERC'].astype(float)

df=df[(df['CREDIT_DEBIT']!='')]
df=df.drop('description',axis=1)

#update the information to transform any prepaid to debit
df['CREDIT_DEBIT']=df['CREDIT_DEBIT'].apply(lambda x:'Debit' if ('Prepaid'==x) or ('Debit'==x) else 'Credit')
df['REGIONALITY'][(df['REGIONALITY']=='Inter-region')]='Interregional'
df['REGIONALITY'][(df['REGIONALITY']=='Intra-region')]='Intraregional'
df['REGIONALITY'][(df['REGIONALITY']=='Domestic')]='Domestic'

#drop countries
df=df.drop(['ISSUER_COUNTRY','MERCHANT_COUNTRY'],axis=1)

C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [26]:
df.REGIONALITY.unique()

array(['Intraregional', 'Interregional', 'Domestic'], dtype=object)

In [27]:
#df=df.groupby(['CONTRACT_ID','CARD_SCHEME','SECURITY_LEVEL','CREDIT_DEBIT','CONSUMER_CORPORATE','ROUNDED_TRXN_AMOUNT','BIN_DESC','REGIONALITY','EEA_DESIGNATION'],as_index=False)[['AIB_SF_BPS','sf_rate_amount','sf_rate_perc']].mean()
df=df.groupby(['CARD_SCHEME','SECURITY_LEVEL','CREDIT_DEBIT','CONSUMER_CORPORATE','BIN_DESC','REGIONALITY','EEA_DESIGNATION'],as_index=False)[['AIB_SF_BPS','AIB_SF_RATE_AMOUNT','AIB_SF_RATE_PERC']].mean()

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49 entries, 0 to 48
Data columns (total 10 columns):
CARD_SCHEME           49 non-null object
SECURITY_LEVEL        49 non-null object
CREDIT_DEBIT          49 non-null object
CONSUMER_CORPORATE    49 non-null object
BIN_DESC              49 non-null object
REGIONALITY           49 non-null object
EEA_DESIGNATION       49 non-null object
AIB_SF_BPS            49 non-null float64
AIB_SF_RATE_AMOUNT    49 non-null float64
AIB_SF_RATE_PERC      49 non-null float64
dtypes: float64(3), object(7)
memory usage: 4.2+ KB


In [29]:
df.head()

,CARD_SCHEME,SECURITY_LEVEL,CREDIT_DEBIT,CONSUMER_CORPORATE,BIN_DESC,REGIONALITY,EEA_DESIGNATION,AIB_SF_BPS,AIB_SF_RATE_AMOUNT,AIB_SF_RATE_PERC
0,Maestro,NON-SECURE,Credit,Consumer,CLASSIC,Intraregional,EEA,0.014345,0.03,0.00059
1,Maestro,NON-SECURE,Credit,Consumer,CLASSIC,Intraregional,NON-EEA,0.022996,0.03,0.00464
2,MasterCard,NON-SECURE,Credit,Commercial,CLASSIC,Domestic,EEA,0.005593,0.03,0.00099
3,MasterCard,NON-SECURE,Credit,Commercial,CLASSIC,Interregional,NON-EEA,0.020288,0.09,0.00585
4,MasterCard,NON-SECURE,Credit,Commercial,CLASSIC,Intraregional,EEA,0.008340,0.03,0.00099


In [30]:
##master_df=pd.merge(bb1,df,on=['CONTRACT_ID','ISSUER_COUNTRY','MERCHANT_COUNTRY','CARD_SCHEME','SECURITY_LEVEL','CREDIT_DEBIT','CONSUMER_CORPORATE','ROUNDED_TRXN_AMOUNT','BIN_DESC'],how='left')
##master_df=pd.merge(bb1,df,on=['REGIONALITY','CONTRACT_ID','CARD_SCHEME','SECURITY_LEVEL','CREDIT_DEBIT','CONSUMER_CORPORATE','ROUNDED_TRXN_AMOUNT','BIN_DESC','EEA_DESIGNATION'],how='left')
master_df=pd.merge(bb1,df,on=['REGIONALITY','CARD_SCHEME','SECURITY_LEVEL','CREDIT_DEBIT','CONSUMER_CORPORATE','BIN_DESC','EEA_DESIGNATION'],how='left')

In [31]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40597 entries, 0 to 40596
Data columns (total 31 columns):
TRANSACTION_REF              40597 non-null object
ADDITIONAL_REFERENCE_2       40597 non-null float64
PAYMENTREFERENCE             40597 non-null float64
BB_TRANSACTION_TYPE          40597 non-null object
CONTRACT_ID                  40597 non-null float64
BIN_DESC                     40597 non-null object
ORDERID                      40597 non-null float64
MONTH                        40597 non-null float64
DAY                          40597 non-null float64
YEAR                         40597 non-null float64
PERIOD                       40597 non-null object
ISSUER_COUNTRY               40597 non-null object
MERCHANT_COUNTRY             40597 non-null object
TRANSACTION_TYPE             40597 non-null object
CARD_SCHEME                  40597 non-null object
CREDIT_DEBIT                 40597 non-null object
CONSUMER_CORPORATE           40597 non-null object
SECURITY_LEVEL   

In [32]:
master_df['EUROPE_ISSUING_SUBREGION']=master_df['EUROPE_ISSUING_SUBREGION'].fillna('NON-EEA')

In [39]:
master_df['AIB_SF_COST']=(master_df['FLOW']*master_df['AIB_SF_RATE_PERC'])+master_df['AIB_SF_RATE_AMOUNT']

In [40]:
master_df=master_df[[
       'TRANSACTION_REF',
        'ADDITIONAL_REFERENCE_2',
        'PAYMENTREFERENCE',
        'BB_TRANSACTION_TYPE',
        'CONTRACT_ID',
        'BIN_DESC',
        'ORDERID',
        'MONTH',
        'DAY',
        'YEAR',
        'PERIOD',
        'ISSUER_COUNTRY',
        'MERCHANT_COUNTRY',
        'TRANSACTION_TYPE',
        'CARD_SCHEME',
        'CREDIT_DEBIT',
        'CONSUMER_CORPORATE',
        'SECURITY_LEVEL',
        'REGIONALITY',
        'EUROPE_ISSUING_SUBREGION',
        'EUROPE_CLEARING_SUBREGION',
        'EEA_DESIGNATION',
        'FLOW',
        'SF_FIXED_FEE',
        'SF_PERC_FEE',
        'AIB_SF_RATE_AMOUNT',
        'AIB_SF_RATE_PERC',
        'SCHEME_TOTAL_FEE_EUR',
        'AIB_SF_COST',
        'BB_SF_BPS']]

In [41]:
master_df['AIB_SF_BPS']=master_df['AIB_SF_COST']/master_df['FLOW']

C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [43]:
master_df.head()

,TRANSACTION_REF,ADDITIONAL_REFERENCE_2,PAYMENTREFERENCE,BB_TRANSACTION_TYPE,CONTRACT_ID,BIN_DESC,ORDERID,MONTH,DAY,YEAR,...,EEA_DESIGNATION,FLOW,SF_FIXED_FEE,SF_PERC_FEE,AIB_SF_RATE_AMOUNT,AIB_SF_RATE_PERC,SCHEME_TOTAL_FEE_EUR,AIB_SF_COST,BB_SF_BPS,AIB_SF_BPS
0,225802414299,2.258024e+11,2.258024e+11,Sale,6891.0,CLASSIC,16621009.0,2.0,17.0,2018.0,...,EEA,8.65,0.004896,0.000299,0.014707,0.00020,0.007482,0.016437,0.000865,0.001900
1,225958428498,2.259584e+11,2.259584e+11,Sale,6891.0,CLASSIC,16727950.0,3.0,13.0,2018.0,...,EEA,8.65,0.004777,0.000299,0.014707,0.00020,0.007363,0.016437,0.000851,0.001900
2,225958428479,2.259584e+11,2.259584e+11,Sale,6891.0,CLASSIC,16727861.0,3.0,13.0,2018.0,...,EEA,8.65,0.022242,0.001885,0.030000,0.00059,0.038548,0.035103,0.004456,0.004058
3,225934411851,2.259344e+11,2.259344e+11,Sale,6891.0,CLASSIC,16706067.0,3.0,9.0,2018.0,...,NON-EEA,8.65,0.196247,0.005460,0.090000,0.00545,0.243477,0.137143,0.028148,0.015855
4,225958428399,2.259584e+11,2.259584e+11,Sale,6891.0,CLASSIC,16727242.0,3.0,13.0,2018.0,...,EEA,9.15,0.022242,0.001460,0.030000,0.00059,0.035602,0.035398,0.003891,0.003869


In [45]:
master_df['SF_DIFF_%']=(master_df['BB_SF_BPS']-master_df['AIB_SF_BPS'])/master_df['AIB_SF_BPS']

In [46]:
master_df['SF_SIGNAL']=''
master_df['SF_SIGNAL']=master_df['SF_DIFF_%'].apply(lambda x:'Signal' if (x>=.05) else '')

In [48]:
master_df.head()

,TRANSACTION_REF,ADDITIONAL_REFERENCE_2,PAYMENTREFERENCE,BB_TRANSACTION_TYPE,CONTRACT_ID,BIN_DESC,ORDERID,MONTH,DAY,YEAR,...,SF_FIXED_FEE,SF_PERC_FEE,AIB_SF_RATE_AMOUNT,AIB_SF_RATE_PERC,SCHEME_TOTAL_FEE_EUR,AIB_SF_COST,BB_SF_BPS,AIB_SF_BPS,SF_DIFF_%,SF_SIGNAL
0,225802414299,2.258024e+11,2.258024e+11,Sale,6891.0,CLASSIC,16621009.0,2.0,17.0,2018.0,...,0.004896,0.000299,0.014707,0.00020,0.007482,0.016437,0.000865,0.001900,-0.544806,
1,225958428498,2.259584e+11,2.259584e+11,Sale,6891.0,CLASSIC,16727950.0,3.0,13.0,2018.0,...,0.004777,0.000299,0.014707,0.00020,0.007363,0.016437,0.000851,0.001900,-0.552046,
2,225958428479,2.259584e+11,2.259584e+11,Sale,6891.0,CLASSIC,16727861.0,3.0,13.0,2018.0,...,0.022242,0.001885,0.030000,0.00059,0.038548,0.035103,0.004456,0.004058,0.098124,Signal
3,225934411851,2.259344e+11,2.259344e+11,Sale,6891.0,CLASSIC,16706067.0,3.0,9.0,2018.0,...,0.196247,0.005460,0.090000,0.00545,0.243477,0.137143,0.028148,0.015855,0.775358,Signal
4,225958428399,2.259584e+11,2.259584e+11,Sale,6891.0,CLASSIC,16727242.0,3.0,13.0,2018.0,...,0.022242,0.001460,0.030000,0.00059,0.035602,0.035398,0.003891,0.003869,0.005749,


In [58]:
print 'total transactions'
print float(master_df['TRANSACTION_REF'].count())
print 'total signals'
print float(master_df['TRANSACTION_REF'][(master_df['SF_SIGNAL']=='Signal')].count())
print 'percentage'
print float(master_df['TRANSACTION_REF'][(master_df['SF_SIGNAL']=='Signal')].count())/float(master_df['TRANSACTION_REF'].count())


total transactions
40597.0
total signals
10949.0
percentage
0.269699731507


In [60]:
master_df.to_csv('d:\\djohnson\\Desktop\\bambora.csv')

In [61]:
#create a text/plain message
msg=email.mime.Multipart.MIMEMultipart()
msg['Subject'] ='AIB to Bambora Scheme Fee Comparison Test'
msg['From']='dale.johnson@epay.ingenico.com'
msg['To']='ariel.Setton@epay.ingenico.com';'dale.johnson@epay.ingenico.com'

#The main body is just another attachment
body = email.mime.Text.MIMEText(
"""
Hello Ariel,
This message is being sent to you from the automated AIB to Bambora SF automated application.
The data looks at only those merchants and lanes that moved directly from AIB to Bambora.
However the system does have the ability to look at all lanes and compare scheme fee costs 
to see what the cost on a lane would be if it was processed with AIB. The only change that
would need to be made is telling the application to do a global comparison instead of the localized one.

Please see below the summary data from the most recent application run. Again the number 40k is because the 
comparison has been localized to only those merchants that were directly moved from AIB to Bambora.

total transactions:
40597.0
total signals:
10949.0
signal percentage to total:
0.269699731507""")
msg.attach(body)

#PDF attachment
filename='D:\\djohnson\\Desktop\\bambora.csv'
fp=open(filename,'rb')
att = email.mime.application.MIMEApplication(fp.read(),_subtype="xlsb")
fp.close()
att.add_header('Content-Disposition','attachment',filename=filename)
msg.attach(att)

In [62]:
#simple mail transfer protocol via the outlook server
smtpObj=smtplib.SMTP('smtp-mail.outlook.com',587)

In [63]:
#establishing communications with the simple mail transfer protocl email server.
#ehlo is simply the script calling to the server and saying hello. The resulting 250 integeer means the 
#call was successful
smtpObj.ehlo()

(250,
 'AM6PR07CA0003.outlook.office365.com Hello [208.49.188.174]\nSIZE 157286400\nPIPELINING\nDSN\nENHANCEDSTATUSCODES\nSTARTTLS\n8BITMIME\nBINARYMIME\nCHUNKING\nSMTPUTF8')

In [64]:
#the starttls enables the tls encrpytion for the connection
smtpObj.starttls()

(220, '2.0.0 SMTP server ready')

In [65]:
#the login information for the sender
smtpObj.login('dale.johnson@epay.ingenico.com','Kirky2016!')

(235,
 '2.7.0 Authentication successful target host VI1PR07MB1312.eurprd07.prod.outlook.com')

In [66]:
#send mail to recipients with message and attachment
smtpObj.sendmail('dale.johnson@epay.ingenico.com',['ariel.Setton@epay.ingenico.com','dale.johnson@epay.ingenico.com'], msg.as_string())

{}

In [67]:
smtpObj.quit()

(221, '2.0.0 Service closing transmission channel')